In [165]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Tuple
import warnings
warnings.filterwarnings("ignore")

In [166]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [167]:

def preprocess_data() -> pd.DataFrame:
    dados = pd.read_csv("super_loja.csv", sep=";", encoding="latin-1")
    dados.dropna(inplace=True)

   
    columns_to_convert = ['Vendas', 'Desconto', 'Lucro', 'longitude', 'latitude']
    dados[columns_to_convert] = dados[columns_to_convert].apply(lambda col: col.str.replace(',', '.').astype(float))

    
    dados['Quantidade'] = dados['Quantidade'].astype(int)

  
    dados['Data do pedido'] = pd.to_datetime(dados['Data do pedido'], format='%d/%m/%Y')

    return dados


def encode_and_scale(data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    encoder = OneHotEncoder(sparse=False)
    categorical_columns_encoded = encoder.fit_transform(data[['Categoria', 'Sub-categoria']])

    scaler = StandardScaler()
    numerical_columns_scaled = scaler.fit_transform(data[['Vendas', 'Quantidade', 'Desconto', 'Lucro']])

    X = np.concatenate((numerical_columns_scaled, categorical_columns_encoded), axis=1)
    y = data['Vendas'].astype(float)

    return X, y


def build_and_train_model(X: np.ndarray, y: np.ndarray) -> tf.keras.models.Sequential:
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=256, activation='relu', input_shape=(X.shape[1],)))
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dense(units=1, activation='linear'))
    model.compile(optimizer='Adam', loss='mean_squared_error')
    model.fit(X, y, epochs=100, batch_size=64, validation_split=0.2)
    return model


def generate_future_predictions(model: tf.keras.models.Sequential, X: np.ndarray, data: pd.DataFrame) -> Tuple[List[datetime], List[float], List[str], List[str], List[int]]:
    end_date = datetime.now() + timedelta(days=3 * 365)
    date_rng = pd.date_range(start=datetime.now(), end=end_date, freq='M')

    future_dates = []
    future_predictions = []
    cities = []
    states = []
    order_counts = []

    num_predictions = 3 * 12 

    for i in range(num_predictions):
       
        last_X = X[-1].reshape(1, -1)
        last_X = (last_X - X.mean(axis=0)) / X.std(axis=0)

        prediction = model.predict(last_X)

       
        prediction = (prediction * X.std(axis=0)) + X.mean(axis=0)

       
        real_value = prediction[0][0]

        future_predictions.append(real_value)
        future_dates.append(date_rng[i])

       
        last_X = (last_X * X.std(axis=0)) + X.mean(axis=0)
        X = np.concatenate((X, last_X), axis=0)

        cities.append(data['Cidade'].iloc[i])
        states.append(data['Estado'].iloc[i])

        order_count = data[data['Cidade'] == cities[-1]].shape[0]
        order_counts.append(order_count)

    return future_dates, future_predictions, cities, states, order_counts


def find_nearest_date(date, dates):
    return dates[np.argmin(np.abs(dates - date))]


data = preprocess_data()
X, y = encode_and_scale(data)
model = build_and_train_model(X, y)
future_dates, future_predictions, cities, states, order_counts = generate_future_predictions(model, X, data)


target_sales = 100000 

for date, prediction, city, state, order_count in zip(future_dates, future_predictions, cities, states, order_counts):
    nearest_date = find_nearest_date(date, data['Data do pedido'])
    real_value = data[data['Data do pedido'] == nearest_date]['Vendas'].values[0]

    if real_value >= target_sales:
        print(f"Data: {date.strftime('%Y-%m-%d')}, Cidade: {city}, Estado: {state}, Previsão: {prediction:.2f}, Valor Real: {real_value:.2f}, Pedidos: {order_count} - Boa cidade para abrir uma nova loja!")
    else:
        print(f"Data: {date.strftime('%Y-%m-%d')}, Cidade: {city}, Estado: {state}, Previsão: {prediction:.2f}, Valor Real: {real_value:.2f}, Pedidos: {order_count}")




Epoch 1/100
24/24 [==============================] - 1s 8ms/step - loss: 19872230.0000 - val_loss: 9219430.0000
Epoch 2/100
24/24 [==============================] - 0s 2ms/step - loss: 19747792.0000 - val_loss: 9102701.0000
Epoch 3/100
24/24 [==============================] - 0s 3ms/step - loss: 19323516.0000 - val_loss: 8798178.0000
Epoch 4/100
24/24 [==============================] - 0s 3ms/step - loss: 18249080.0000 - val_loss: 8170896.5000
Epoch 5/100
24/24 [==============================] - 0s 3ms/step - loss: 16139458.0000 - val_loss: 7188104.0000
Epoch 6/100
24/24 [==============================] - 0s 3ms/step - loss: 13088805.0000 - val_loss: 5929424.5000
Epoch 7/100
24/24 [==============================] - 0s 2ms/step - loss: 9786229.0000 - val_loss: 4666977.0000
Epoch 8/100
24/24 [==============================] - 0s 3ms/step - loss: 6885287.5000 - val_loss: 3710232.5000
Epoch 9/100
24/24 [==============================] - 0s 3ms/step - loss: 4818581.0000 - val_loss: 3140708.

In [168]:

data = preprocess_data()
X, y = encode_and_scale(data)
model = build_and_train_model(X, y)
future_dates, future_predictions, cities, states, order_counts = generate_future_predictions(model, X, data)


target_sales = 100000


processed_cities = set()

for date, prediction, city, state, order_count in zip(future_dates, future_predictions, cities, states, order_counts):
    if city not in processed_cities:
        processed_cities.add(city)

     
        nearest_date = find_nearest_date(date, data['Data do pedido'])
        real_value = data[data['Data do pedido'] == nearest_date]['Vendas'].values[0]

      
        probability = real_value / target_sales

        
        months_until_good_probability = 0
        while probability < 1.0:
            
            if date < data['Data do pedido'].max():
                months_until_good_probability += 1
                date = find_nearest_date(date + timedelta(days=30), data['Data do pedido'])
                nearest_date = find_nearest_date(date, data['Data do pedido'])
                real_value = data[data['Data do pedido'] == nearest_date]['Vendas'].values[0]
                probability = real_value / target_sales
            else:
                break

        print(f"Data: {date.strftime('%Y-%m-%d')}, Cidade: {city}, Estado: {state}, Previsão: {prediction:.2f}, Valor Real: {real_value:.2f}, Pedidos: {order_count}, Probabilidade: {probability:.2%}")

        if probability >= 1.0:
            print("Boa cidade para abrir uma nova loja!")
        else:
            print(f"Não recomendado abrir uma nova loja. Esperar pelo menos {months_until_good_probability} meses.")




Epoch 1/100
24/24 [==============================] - 1s 7ms/step - loss: 19872066.0000 - val_loss: 9217522.0000
Epoch 2/100
24/24 [==============================] - 0s 2ms/step - loss: 19744060.0000 - val_loss: 9096115.0000
Epoch 3/100
24/24 [==============================] - 0s 3ms/step - loss: 19286414.0000 - val_loss: 8774441.0000
Epoch 4/100
24/24 [==============================] - 0s 3ms/step - loss: 18148206.0000 - val_loss: 8122755.5000
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 16054775.0000 - val_loss: 7059939.5000
Epoch 6/100
24/24 [==============================] - 0s 4ms/step - loss: 12835440.0000 - val_loss: 5794901.5000
Epoch 7/100
24/24 [==============================] - 0s 3ms/step - loss: 9552804.0000 - val_loss: 4540966.0000
Epoch 8/100
24/24 [==============================] - 0s 3ms/step - loss: 6613746.0000 - val_loss: 3647917.2500
Epoch 9/100
24/24 [==============================] - 0s 3ms/step - loss: 4789502.5000 - val_loss: 3101742.

In [169]:
from datetime import timedelta

data = preprocess_data()
X, y = encode_and_scale(data)
model = build_and_train_model(X, y)
future_dates, future_predictions, cities, states, order_counts = generate_future_predictions(model, X, data)

target_sales = 100000

processed_cities = set()

results = []

for date, prediction, city, state, order_count in zip(future_dates, future_predictions, cities, states, order_counts):
    if city not in processed_cities:
        nearest_date = find_nearest_date(date, data['Data do pedido'])
        real_value = data[data['Data do pedido'] == nearest_date]['Vendas'].values[0]

        probability = (prediction / real_value) * 100

        months_until_good_probability = 0
        if probability < 100.0:
            while probability < 100.0:
                if date < data['Data do pedido'].max():
                    date = find_nearest_date(date + timedelta(days=30), data['Data do pedido'])
                    nearest_date = find_nearest_date(date, data['Data do pedido'])
                    real_value = data[data['Data do pedido'] == nearest_date]['Vendas'].values[0]
                    probability = (prediction / real_value) * 100
                    months_until_good_probability += 1
                else:
                    break

        result = f"Data: {date.strftime('%Y-%m-%d')}, Cidade: {city}, Estado: {state}, Previsão: {prediction:.2f}, Valor Real: {real_value:.2f}, Pedidos: {order_count}, Probabilidade: {probability:.2f}%"

        if probability >= 100.0:
            result += " - Boa cidade para abrir uma nova loja!"
        else:
            if months_until_good_probability == 0:
                result += " - Não recomendado abrir uma nova loja imediatamente."
            else:
                result += f" - Não recomendado abrir uma nova loja. Esperar pelo menos {months_until_good_probability} meses."

        results.append(result)
        processed_cities.add(city)


for result in results:
    print(result)




Epoch 1/100
24/24 [==============================] - 1s 11ms/step - loss: 19874180.0000 - val_loss: 9222545.0000
Epoch 2/100
24/24 [==============================] - 0s 3ms/step - loss: 19752592.0000 - val_loss: 9112200.0000
Epoch 3/100
24/24 [==============================] - 0s 3ms/step - loss: 19343510.0000 - val_loss: 8811506.0000
Epoch 4/100
24/24 [==============================] - 0s 3ms/step - loss: 18279490.0000 - val_loss: 8208992.5000
Epoch 5/100
24/24 [==============================] - 0s 4ms/step - loss: 16274364.0000 - val_loss: 7222155.0000
Epoch 6/100
24/24 [==============================] - 0s 3ms/step - loss: 13450255.0000 - val_loss: 5978918.5000
Epoch 7/100
24/24 [==============================] - 0s 3ms/step - loss: 10052058.0000 - val_loss: 4788148.5000
Epoch 8/100
24/24 [==============================] - 0s 3ms/step - loss: 7167618.5000 - val_loss: 3779854.0000
Epoch 9/100
24/24 [==============================] - 0s 5ms/step - loss: 5019399.5000 - val_loss: 316284

In [ ]:
from collections import defaultdict

def calculate_city_success(city_data):
    total_sales = 0
    total_predictions = 0

    for real_value, prediction, order_count in city_data:
        total_sales += real_value
        total_predictions += prediction

    if total_sales > 0:
        probability = (total_predictions / total_sales) * 100
    else:
        probability = 0

    if probability >= 100:
        recommendation = "Recomendado abrir uma nova loja."
    else:
        recommendation = "Não recomendado abrir uma nova loja imediatamente."

    return probability, recommendation

def group_data_by_state(future_dates, future_predictions, cities, states, order_counts):
    state_data = defaultdict(list)

    for date, prediction, city, state, order_count in zip(future_dates, future_predictions, cities, states, order_counts):
        nearest_date = find_nearest_date(date, data['Data do pedido'])
        real_value = data[data['Data do pedido'] == nearest_date]['Vendas'].values[0]

        state_data[state].append((city, real_value, prediction, order_count))

    return state_data

state_data = group_data_by_state(future_dates, future_predictions, cities, states, order_counts)

for state, cities_data in state_data.items():
    print(f"Estado: {state}")
    for city, real_value, prediction, order_count in cities_data:
        probability, recommendation = calculate_city_success(cities_data)
        print(f"  Cidade: {city} - Probabilidade: {round(probability, 2)}% - {recommendation}")
        print(f"    Valor Real: {real_value:.2f}, Previsão: {round(prediction, 2)}, Pedidos: {order_count}")



In [171]:
dados = pd.read_csv("super_loja.csv", sep=";", encoding="latin-1")
dados.dropna(inplace=True)

In [172]:
lati = dados[['Cidade','longitude','latitude']]


In [173]:
lati = dados[['Cidade','longitude','latitude']]
previsao = pd.DataFrame(lista_dicts)
merged_data = pd.merge(previsao, lati, on='Cidade')

In [174]:

merged_data = pd.merge(previsao, lati, on='Cidade')
merged_data.dropna(inplace=True)
merged_data[['longitude','latitude']] = merged_data[['longitude','latitude']].applymap(lambda k: float(str(k).replace(",",".")))

In [175]:
import plotly.graph_objects as go
import pandas as pd


data = pd.DataFrame({
    'Cidade': ['são paulo', 'brasília', 'santiago', 'goiânia', 'são benedito',
               'pinheiro', 'lençóis paulista', 'abreu e lima', 'natal'],
    'Estado': ['SP', 'DF', 'CH', 'GO', 'CE', 'MA', 'SP', 'PE', 'RN'],
    'Probabilidade': [24.21, 23.80, 23.58, 23.42, 23.31, 23.21, 23.10, 23.00, 22.85],
    'Valor': [12644.66] * 9,
    'Previsao': [3090.55, 3016.98, 2988.76, 2961.22, 2947.77, 2934.50, 2921.37, 2908.40, 2895.59],
    'longitude': [-46.570383, -47.887905, -54.869076, -49.255814, -40.864490, -45.083927, -48.798682, -34.899131, -35.252255],
    'latitude': [-23.567387, -15.794087, -29.191460, -16.673310, -4.044727, -2.522740, -22.597507, -7.904449, -5.750899]
})


normed_values = (data['Valor'] - data['Valor'].min()) / (data['Valor'].max() - data['Valor'].min())


data['text'] = data['Cidade'] + '<br>' + data['Estado'] + '<br>' + \
    'Probabilidade ' + data['Probabilidade'].astype(str) + '<br>' + \
    'Valor ' + data['Valor'].astype(str) + '<br>' + \
    'Previsao ' + data['Previsao'].astype(str)

fig = go.Figure(go.Scattergeo(
    lon=data['longitude'],
    lat=data['latitude'],
    text=data['text'],
    mode='markers',
    marker=dict(
        size=30, 
        opacity=0.8,
        reversescale=True,
        autocolorscale=False,
        symbol='circle',
        line=dict(
            width=1,
            color='rgba(102, 102, 102)'
        ),
        color=normed_values, 
        colorscale='Reds' 
    )
))


for i, row in data.iterrows():
    fig.add_annotation(
        text=row['Cidade'] + ' - ' + row['Estado'],
        x=row['longitude'],
        y=row['latitude'],
        showarrow=False,
    )


fig.update_geos(projection_type='mercator', showland=False) 


fig.update_layout(
    title='Mapa Brasil',
    geo=dict(
        scope='south america',
        showcoastlines=False, 
    )
)


fig.show()



In [176]:

merged_data = pd.merge(previsao, lati, on='Cidade')
merged_data.dropna(inplace=True)
merged_data[['longitude','latitude']] = merged_data[['longitude','latitude']].applymap(lambda k: float(str(k).replace(",",".")))
merged_data

,Cidade,Probabilidade,valor,Previsao,longitude,latitude
0,são paulo,25.11,12644.66,3204.65,-46.570383,-23.567387
1,são paulo,25.11,12644.66,3204.65,-46.570383,-23.567387
2,são paulo,25.11,12644.66,3204.65,-46.570383,-23.567387
3,são paulo,25.11,12644.66,3204.65,-46.570383,-23.567387
4,são paulo,25.11,12644.66,3204.65,-46.570383,-23.567387
...,...,...,...,...,...,...
529,osasco,22.09,12644.66,2793.42,-46.788810,-23.533612
530,osasco,22.09,12644.66,2793.42,-46.788810,-23.533612
531,osasco,22.09,12644.66,2793.42,-46.788810,-23.533612
532,osasco,22.09,12644.66,2793.42,-46.788810,-23.533612


In [177]:
import plotly.express as px
import pandas as pd


# Criar um gráfico de série temporal
fig = px.line(merged_data, x=merged_data['Cidade'], y=['valor', 'Previsao'], title='Real Vs Previsto')
fig.update_xaxes(title='Cidade')
fig.update_yaxes(title='Valor')
fig.show()
